In [16]:
import mido
from genetic_musical_generator.random_genome_to_midi import random_genome, genome2midi
from dataclasses import dataclass

from typing import Optional

In [12]:
outport = mido.open_output('mido_out', virtual=True)

def hear(genome):
    song = genome2midi(genome)
    print(song)
    for m in song.play():
        outport.send(m)

In [10]:
e = random_genome(40)
e

'BUBFDDBFFDSSBFFUUBFBUDBSFSUFUUDBDFDFUDBS'

In [13]:
hear(e)

MidiFile(type=0, ticks_per_beat=480, tracks=[
  MidiTrack([
    Message('note_on', channel=0, note=60, velocity=64, time=800),
    Message('note_off', channel=0, note=60, velocity=64, time=0),
    Message('note_on', channel=0, note=60, velocity=64, time=800),
    Message('note_off', channel=0, note=60, velocity=64, time=0),
    Message('note_on', channel=0, note=60, velocity=64, time=800),
    Message('note_off', channel=0, note=60, velocity=64, time=0)])
])


In [26]:
@dataclass
class Individual:
    genome: str
    rating: Optional[float] = None

In [27]:
pop_size = 10

In [32]:
population = []

In [33]:
while True:
    while len(population)<pop_size:
        population.append(Individual(random_genome(40)))

        for p in population:
            while p.rating is None:
                try:
                    hear(p.genome)
                    p.rating = float(input('Please rate (zero or less to kill): '))
                except ValueError:
                    pass
            print()

        for i in range(len(population)):
            if population[i].rating <= 0:
                del population[i]

MidiFile(type=0, ticks_per_beat=480, tracks=[
  MidiTrack([
    Message('note_on', channel=0, note=60, velocity=64, time=343),
    Message('note_on', channel=0, note=63, velocity=64, time=69),
    Message('note_off', channel=0, note=63, velocity=64, time=69),
    Message('note_off', channel=0, note=60, velocity=64, time=206),
    Message('note_on', channel=0, note=70, velocity=64, time=0),
    Message('note_on', channel=0, note=69, velocity=64, time=0),
    Message('note_on', channel=0, note=69, velocity=64, time=343),
    Message('note_off', channel=0, note=70, velocity=64, time=0),
    Message('note_off', channel=0, note=69, velocity=64, time=0),
    Message('note_off', channel=0, note=69, velocity=64, time=0)])
])
Please rate (zero or less to kill): 1


MidiFile(type=0, ticks_per_beat=480, tracks=[
  MidiTrack([
    Message('note_on', channel=0, note=60, velocity=64, time=0),
    Message('note_on', channel=0, note=59, velocity=64, time=480),
    Message('note_off', channel=0, note=6

Please rate (zero or less to kill): .1








MidiFile(type=0, ticks_per_beat=480, tracks=[
  MidiTrack([
    Message('note_on', channel=0, note=59, velocity=64, time=192),
    Message('note_off', channel=0, note=59, velocity=64, time=0),
    Message('note_on', channel=0, note=60, velocity=64, time=0),
    Message('note_on', channel=0, note=58, velocity=64, time=192),
    Message('note_off', channel=0, note=58, velocity=64, time=288),
    Message('note_off', channel=0, note=60, velocity=64, time=0)])
])
Please rate (zero or less to kill): 0








MidiFile(type=0, ticks_per_beat=480, tracks=[
  MidiTrack([
    Message('note_on', channel=0, note=65, velocity=64, time=3360),
    Message('note_off', channel=0, note=65, velocity=64, time=13440),
    Message('note_on', channel=0, note=65, velocity=64, time=3360),
    Message('note_off', channel=0, note=65, velocity=64, time=480),
    Message('note_on', channel=0, note=61, velocity=64, time=0),
    Message('note_on', channel=0, note=62, ve

KeyboardInterrupt: 